In [1]:
import sys

In [2]:
sys.path.append("../")

In [3]:
from dgs.data.denotation_graph import DenotationGraph, visualize_denotation_graph

In [4]:
dg = DenotationGraph(graph_folder_path="../../../flicker30k_denotation_graph/flicker30k/graph", limit_height=3)

ℹ Reading node.idx file
ℹ Reading node-cap.map file
ℹ Reading node-img.map file
ℹ Reading token.txt file
ℹ Reading the node-tree.txt file


Finding full caption leaf nodes...: 100%|██████████| 621175/621175 [00:00<00:00, 944633.27it/s]


ℹ Limiting the height of the Denotation graph to: 3 and retaining only
the full_caption_leaf_nodes


Finding full caption leaf nodes...: 100%|██████████| 150907/150907 [00:00<00:00, 877937.38it/s]


In [5]:
len(dg.leaf_nodes)

150907

In [6]:
len(dg.full_caption_leaf_nodes)

150907

In [7]:
len(dg.nodes)

585567

In [8]:
%%time
len(dg)

CPU times: user 62.5 ms, sys: 143 µs, total: 62.7 ms
Wall time: 62.3 ms


585567

In [9]:
%%time
nodes = dg.nodes

CPU times: user 68.4 ms, sys: 21 µs, total: 68.5 ms
Wall time: 67.4 ms


In [10]:
import numpy as np
np.random.seed(75252)

In [11]:
node=7500

In [12]:
dg.get_node_info(node)["text"]

'adult wear black clothing standing'

In [13]:
%%time
children = dg.get_descendents(node)

CPU times: user 189 µs, sys: 12 µs, total: 201 µs
Wall time: 204 µs


In [14]:
len(children)

99

In [15]:
children.add(node)

In [16]:
type(nodes)

set

In [17]:
visually_mismatched_images = nodes - children

In [18]:
len(visually_mismatched_images)

585467

In [19]:
from itertools import chain

In [20]:
from tqdm import tqdm

In [ ]:
# visually_mismatched = []
# semantically_mismatched = []
# dg_hard_negatives = []
# positive_sample = []

# for node in tqdm(my_nodes):
#     current_node_info = dg.get_node_info(node)
#     current_image = np.random.choice(current_node_info["images"])
#     current_text = current_node_info["text"]
#     relation_nodes = dg.get_descendents(node)
#     relation_nodes.update(dg.get_ancestors(node))
#     mismatched_nodes = my_nodes - relation_nodes
    
#     mismatched_node = np.random.choice(tuple(mismatched_nodes))
#     mismatched_node_info = dg.get_node_info(mismatched_node)
#     mismatched_image= np.random.choice(mismatched_node_info["images"])
#     mismatched_text= mismatched_node_info["text"]
#     visually_mismatched.append((mismatched_image, current_text))
#     semantically_mismatched.append((current_image, mismatched_text))
#     sibling_nodes = tuple(chain(*dg.get_sibling_nodes(node).values()))
#     hard_negative_node = np.random.choice(sibling_nodes)
#     dg_hard_negatives.append((current_image, dg.get_node_info(hard_negative_node)["text"]))

In [21]:
from multiprocessing import Pool
from collections import namedtuple

In [22]:
def get_data(node):
    Samples = namedtuple("samples", ["positive_sample","visually_mismatched", "semantically_mismatched", "dg_hard_negative"])
    global dg
    current_node_info = dg.get_node_info(node)
    current_image = np.random.choice(current_node_info["images"])
    current_text = current_node_info["text"]
    relation_nodes = dg.get_descendents(node)
    relation_nodes.update(dg.get_ancestors(node))
    mismatched_nodes = my_nodes - relation_nodes
    
    mismatched_node = np.random.choice(tuple(mismatched_nodes))
    mismatched_node_info = dg.get_node_info(mismatched_node)
    mismatched_image= np.random.choice(mismatched_node_info["images"])
    mismatched_text= mismatched_node_info["text"]
    sibling_nodes = tuple(chain(*dg.get_sibling_nodes(node).values()))
    hard_negative_node = np.random.choice(sibling_nodes)
    sample = Samples(
        (current_image, current_text),
        (mismatched_image, current_text),
        (current_image, mismatched_text),
        (current_image, dg.get_node_info(hard_negative_node)["text"])
                    )

In [23]:
nodes = dg.nodes
root_nodes = set([n for n in nodes if dg.is_root_node(n) is True])
my_nodes = nodes - root_nodes

In [24]:
my_nodes = list(my_nodes)

In [26]:
num_workers = 8
pool = Pool(num_workers)
samples = tqdm(pool.map(get_data, my_nodes))

TypeError: unsupported operand type(s) for -: 'list' and 'set'